In [2]:
import pandas as pd
import sqlite3

# Database file (Creates if it doesn't exist)
db_name = "stocks_data.db"

# Connect to SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

In [2]:
from yahooquery import Ticker

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
# df_st_table= pd.read_sql("SELECT * FROM stock_prices", conn)
# df_st_table.count

In [116]:
df_NSE = pd.read_csv('/Users/jayaprakash/Downloads/NSELIST.csv')
df_NSE1 = df_NSE['TICKER']
df_NSE18 = df_NSE1.tolist()
# df_NSE
df_NSE18

df_st_table= pd.read_sql("SELECT distinct ticker FROM stock_prices", conn)
st_list = df_st_table['ticker'].tolist()
missed_tickers =  []
for i in df_NSE18:
    if i not in st_list:
        missed_tickers.append(i)

missed_tickers

subset = missed_tickers[:100]  # Select first 300 elements
subset

dailyprice = []
import time

for i in subset:
    time.sleep(1)
    try:
        tickers = Ticker(i, asynchronous=True)
        df = tickers.history(period='max', interval='1d')
        if not df.empty:
            df['ticker'] = i
            dailyprice.append(df)
    except Exception as e:
        print("error in fetching record {i}:{e}")

df_daily_values = pd.concat(dailyprice)


ValueError: No objects to concatenate

In [117]:
missed_tickers

[]

In [110]:
df_daily_values2 = pd.concat(dailyprice)
df_daily_values2.reset_index(inplace=True)
df_daily_values3 = df_daily_values2.drop(columns=['dividends','splits'], errors='ignore')
df_daily_values3
#1248024

/var/folders/ws/pm1tbbc178114g4lvl7vynp40000gp/T/ipykernel_13456/2945664598.py:1: RuntimeWarning: can't compare datetime.datetime to datetime.date, sort order is undefined for incomparable objects.
  df_daily_values2 = pd.concat(dailyprice)


,symbol,date,open,high,low,close,volume,adjclose,ticker
0,SBICARD.NS,2020-03-16,661.000000,755.000000,656.000000,681.400024,60887005.0,671.316589,SBICARD.NS
1,SBICARD.NS,2020-03-17,691.000000,747.950012,689.000000,730.900024,15145135.0,720.084045,SBICARD.NS
2,SBICARD.NS,2020-03-18,752.099976,769.000000,680.000000,689.200012,8949832.0,679.001160,SBICARD.NS
3,SBICARD.NS,2020-03-19,660.000000,717.000000,650.000000,692.049988,6432046.0,681.808899,SBICARD.NS
4,SBICARD.NS,2020-03-20,706.250000,730.000000,690.250000,724.500000,6492226.0,713.778748,SBICARD.NS
...,...,...,...,...,...,...,...,...,...
494944,ZYDUSWELL.NS,2025-03-20,1624.949951,1667.150024,1624.949951,1650.449951,71909.0,1650.449951,ZYDUSWELL.NS
494945,ZYDUSWELL.NS,2025-03-21,1661.550049,1676.949951,1645.349976,1668.800049,16485.0,1668.800049,ZYDUSWELL.NS
494946,ZYDUSWELL.NS,2025-03-24,1668.800049,1672.949951,1649.599976,1667.599976,23153.0,1667.599976,ZYDUSWELL.NS
494947,ZYDUSWELL.NS,2025-03-25,1675.000000,1688.750000,1651.000000,1680.150024,33812.0,1680.150024,ZYDUSWELL.NS


In [67]:
df_st_table= pd.read_sql("SELECT ticker, max(date) as date FROM stock_prices group by ticker", conn)
df_st_table['date'] = pd.to_datetime(df_st_table['date'])
# Calculate days from today (change 'today' as needed)
#df_st_table['Days_From_Today'] = (pd.Timestamp.today().normalize() - df_st_table['date']).dt.days + str('d')
df_st_table['Days_From_Today'] = df_st_table['date'].apply(
    lambda x: str((pd.Timestamp.today().normalize() - x).days) + 'd'
)
df_NSE_st = df_NSE1.merge(df_st_table, left_on='TICKER', right_on='ticker', how='left')
df_NSE_st = df_NSE_st.assign(Days_From_Today=df_NSE_st['Days_From_Today'].fillna("max"))
df_NSE_st_filtered = df_NSE_st[df_NSE_st['Days_From_Today'] == 'max']
df_NSE_st

,TICKER,ticker,date,Days_From_Today
0,20MICRONS.NS,20MICRONS.NS,2025-03-21,3d
1,21STCENMGM.NS,21STCENMGM.NS,2025-03-21,3d
2,360ONE.NS,360ONE.NS,2025-03-21,3d
3,3IINFOLTD.NS,3IINFOLTD.NS,2025-03-21,3d
4,3MINDIA.NS,3MINDIA.NS,2025-03-21,3d
...,...,...,...,...
2085,ZOTA.NS,NaN,NaT,max
2086,ZUARI.NS,NaN,NaT,max
2087,ZUARIIND.NS,NaN,NaT,max
2088,ZYDUSLIFE.NS,NaN,NaT,max


In [54]:
df_NSE_st = df_NSE_st.head(2000)
df_NSE_st.to_csv('/Users/jayaprakash/Desktop/eash_file11.csv')

In [ ]:
df_st_table= pd.read_sql("SELECT distinct ticker FROM stock_prices", conn)
st_list = df_st_table['ticker'].tolist()
missed_tickers =  []
for i in df_list_NSE:
    if i not in st_list:
        missed_tickers.append(i)

missed_tickers

subset = missed_tickers[:600]  # Select first 300 elements
subset

In [ ]:
tickers = Ticker('20MICRONS.NS', asynchronous=True)
df = tickers.history(period='max', interval='1d')

In [45]:
dailyprice = []
import time

for index, row in df_NSE_st.iterrows():
    i = row['TICKER']
    k = row['Days_From_Today']
    time.sleep(1)
    try:
        tickers = Ticker(i, asynchronous=True)
        df = tickers.history(period=k, interval='1d')
        if not df.empty:
            df['ticker'] = i
            dailyprice.append(df)
    except Exception as e:
        print("error in fetching record {i}:{e}")

df_daily_values = pd.concat(dailyprice)


,TICKER,ticker,date,Days_From_Today
0,20MICRONS.NS,20MICRONS.NS,2025-03-21,2d
1,21STCENMGM.NS,21STCENMGM.NS,2025-03-21,2d
2,360ONE.NS,360ONE.NS,2025-03-21,2d
3,3IINFOLTD.NS,3IINFOLTD.NS,2025-03-21,2d
4,3MINDIA.NS,3MINDIA.NS,2025-03-21,2d
...,...,...,...,...
395,COUNCODOS.NS,COUNCODOS.NS,2025-03-21,2d
396,CRAFTSMAN.NS,CRAFTSMAN.NS,2025-03-21,2d
397,CREATIVE.NS,CREATIVE.NS,2025-03-21,2d
398,CREATIVEYE.NS,CREATIVEYE.NS,2025-03-21,2d


In [46]:
dailyprice = []
import time

for index, row in df_NSE_st.iterrows():
    i = row['TICKER']
    k = row['Days_From_Today']
    time.sleep(1)
    try:
        tickers = Ticker(i, asynchronous=True)
        df = tickers.history(period=k, interval='1d')
        if not df.empty:
            df['ticker'] = i
            dailyprice.append(df)
    except Exception as e:
        print("error in fetching record {i}:{e}")

df_daily_values = pd.concat(dailyprice)


error in fetching record {i}:{e}
error in fetching record {i}:{e}
error in fetching record {i}:{e}
error in fetching record {i}:{e}
error in fetching record {i}:{e}
error in fetching record {i}:{e}


In [ ]:
dailyprice = []
import time

for index, row in df_NSE_st.iterrows():
    i = row['TICKER']
    k = row['Days_From_Today']
    time.sleep(1)
    try:
        tickers = Ticker(i, asynchronous=True)
        df = tickers.history(period=k, interval='1d')
        if not df.empty:
            df['ticker'] = i
            dailyprice.append(df)
    except Exception as e:
        print("error in fetching record {i}:{e}")

df_daily_values = pd.concat(dailyprice)


In [68]:
df_daily_values2 = pd.concat(dailyprice)
df_daily_values2.reset_index(inplace=True)
df_daily_values2

,symbol,date,open,high,low,close,volume,adjclose,ticker,dividends
0,20MICRONS.NS,2025-03-20,202.000000,204.949997,200.419998,201.110001,75022.0,201.110001,20MICRONS.NS,NaN
1,20MICRONS.NS,2025-03-21,202.649994,205.710007,201.490005,204.179993,68420.0,204.179993,20MICRONS.NS,NaN
2,21STCENMGM.NS,2025-03-20,61.500000,63.849998,61.500000,62.439999,5768.0,62.439999,21STCENMGM.NS,NaN
3,21STCENMGM.NS,2025-03-21,62.439999,63.680000,62.349998,63.500000,16853.0,63.500000,21STCENMGM.NS,NaN
4,360ONE.NS,2025-03-20,923.799988,930.000000,913.500000,926.599976,554710.0,926.599976,360ONE.NS,NaN
...,...,...,...,...,...,...,...,...,...,...
13449,CREATIVE.NS,2025-03-21,662.049988,690.000000,662.000000,673.900024,36466.0,673.900024,CREATIVE.NS,NaN
13450,CREATIVEYE.NS,2025-03-20,6.030000,6.240000,6.030000,6.150000,25435.0,6.150000,CREATIVEYE.NS,NaN
13451,CREATIVEYE.NS,2025-03-21,6.440000,6.440000,5.950000,6.250000,22123.0,6.250000,CREATIVEYE.NS,NaN
13452,CREDITACC.NS,2025-03-20,958.500000,983.599976,942.650024,971.549988,858637.0,971.549988,CREDITACC.NS,NaN


In [51]:
# df_daily_values = df_daily_values#.drop(columns=['dividends','splits'], errors='ignore')
# #df_daily_values = df_daily_values.drop_duplicates(subset=['ticker', 'date'])

# df_daily_values.reset_index(drop=True, inplace=True)
df_daily_values3 = df_daily_values2.drop(columns=['dividends','splits'], errors='ignore')
df_daily_values3

,symbol,date,open,high,low,close,volume,adjclose,ticker
0,20MICRONS.NS,2025-03-20,202.000000,204.949997,200.419998,201.110001,75022.0,201.110001,20MICRONS.NS
1,20MICRONS.NS,2025-03-21,202.649994,205.710007,201.490005,204.179993,68420.0,204.179993,20MICRONS.NS
2,21STCENMGM.NS,2025-03-20,61.500000,63.849998,61.500000,62.439999,5768.0,62.439999,21STCENMGM.NS
3,21STCENMGM.NS,2025-03-21,62.439999,63.680000,62.349998,63.500000,16853.0,63.500000,21STCENMGM.NS
4,360ONE.NS,2025-03-20,923.799988,930.000000,913.500000,926.599976,554710.0,926.599976,360ONE.NS
...,...,...,...,...,...,...,...,...,...
13449,CREATIVE.NS,2025-03-21,662.049988,690.000000,662.000000,673.900024,36466.0,673.900024,CREATIVE.NS
13450,CREATIVEYE.NS,2025-03-20,6.030000,6.240000,6.030000,6.150000,25435.0,6.150000,CREATIVEYE.NS
13451,CREATIVEYE.NS,2025-03-21,6.440000,6.440000,5.950000,6.250000,22123.0,6.250000,CREATIVEYE.NS
13452,CREDITACC.NS,2025-03-20,958.500000,983.599976,942.650024,971.549988,858637.0,971.549988,CREDITACC.NS


In [111]:
for _, row in df_daily_values3.iterrows():
    try:
        cursor.execute("""
            INSERT INTO stock_prices (symbol, date, open, high, low, close, volume, adjclose, ticker)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            ON CONFLICT(ticker, date) DO UPDATE SET
                open = excluded.open,
                high = excluded.high,
                low = excluded.low,
                close = excluded.close,
                volume = excluded.volume,
                adjclose = excluded.adjclose;
        """, tuple(row))
    except Exception as e:
        print(f"Error inserting data: {e}")

/var/folders/ws/pm1tbbc178114g4lvl7vynp40000gp/T/ipykernel_13456/4254892763.py:3: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute("""
/var/folders/ws/pm1tbbc178114g4lvl7vynp40000gp/T/ipykernel_13456/4254892763.py:3: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute("""


In [7]:
cursor.execute("""CREATE TABLE stock_prices_backup_new (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    symbol TEXT,
    date TEXT NOT NULL,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    volume INTEGER,
    adjclose REAL,
    ticker TEXT NOT NULL,
    UNIQUE (ticker, date)  -- Enforcing uniqueness
);
""")
conn.commit()  # Commit the changes


In [ ]:
ALTER TABLE stock_prices_backup 
ADD CONSTRAINT unique_ticker_date UNIQUE (ticker, date);


In [10]:
df_st_table= pd.read_sql("SELECT * FROM stock_prices", conn)
df_daily_values3 = df_st_table.drop(columns=['dividends','splits'], errors='ignore')
df_daily_values3

,symbol,date,open,high,low,close,volume,adjclose,ticker
0,20MICRONS.NS,2008-10-06,40.000000,40.000000,15.800000,16.775000,23501600.0,14.652864,20MICRONS.NS
1,20MICRONS.NS,2008-10-07,16.000000,19.000000,13.925000,15.025000,9113400.0,13.124249,20MICRONS.NS
2,20MICRONS.NS,2008-10-08,14.000000,14.600000,12.550000,13.250000,2464384.0,11.573796,20MICRONS.NS
3,20MICRONS.NS,2008-10-10,12.450000,12.450000,10.825000,11.600000,1207928.0,10.132533,20MICRONS.NS
4,20MICRONS.NS,2008-10-13,12.150000,13.300000,11.650000,12.325000,898692.0,10.765815,20MICRONS.NS
...,...,...,...,...,...,...,...,...,...
5736814,ZYDUSWELL.NS,2025-03-20,1624.949951,1667.150024,1624.949951,1650.449951,71909.0,1650.449951,ZYDUSWELL.NS
5736815,ZYDUSWELL.NS,2025-03-21,1661.550049,1676.949951,1645.349976,1668.800049,16485.0,1668.800049,ZYDUSWELL.NS
5736816,ZYDUSWELL.NS,2025-03-24,1668.800049,1672.949951,1649.599976,1667.599976,23153.0,1667.599976,ZYDUSWELL.NS
5736817,ZYDUSWELL.NS,2025-03-25,1675.000000,1688.750000,1651.000000,1680.150024,33812.0,1680.150024,ZYDUSWELL.NS


In [11]:
for _, row in df_daily_values3.iterrows():
    try:
        cursor.execute("""
            INSERT INTO stock_prices_backup_new (symbol, date, open, high, low, close, volume, adjclose, ticker)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            ON CONFLICT(ticker, date) DO UPDATE SET
                open = excluded.open,
                high = excluded.high,
                low = excluded.low,
                close = excluded.close,
                volume = excluded.volume,
                adjclose = excluded.adjclose;
        """, tuple(row))
    except Exception as e:
        print(f"Error inserting data: {e}")

In [ ]:
# import sqlite3

# # Connect to SQLite database
# conn = sqlite3.connect("stocks_data.db")
# cursor = conn.cursor()

# # Run PRAGMA to check the table schema
# cursor.execute("PRAGMA table_info(stock_prices);")

# # Fetch and print the result
# columns = cursor.fetchall()
# for col in columns:
#     print(col)

# conn.close()


In [ ]:
# import sqlite3

# conn = sqlite3.connect("stocks_data.db")
# cursor = conn.cursor()

# # Add unique constraint manually
# cursor.execute("CREATE UNIQUE INDEX IF NOT EXISTS idx_ticker_date ON stock_prices (ticker, date);")

# conn.commit()

# print("Unique index on (ticker, date) added successfully!")


In [ ]:
1️⃣ Basic Data Exploration
df.shape → Check the number of rows and columns.
df.info() → Get column data types and missing values.
df.describe() → Summary statistics for numerical columns.
df.head() / df.tail() → View first/last few rows.

2️⃣ Handling Missing Values
df.isnull().sum() → Count missing values in each column.
df.dropna() → Remove missing values.
df.fillna(value) → Impute missing values with a specific value or method.

3️⃣ Data Type and Uniqueness Check
df.dtypes → Identify column data types.
df.nunique() → Count unique values per column.
df.duplicated().sum() → Find duplicate rows.
df.duplicated().sum() → Count duplicate rows.

4️⃣ Statistical Analysis
df.corr() → Find correlation between numerical columns.
df.skew() → Check skewness of distributions.
df.kurt() → Check kurtosis.

5️⃣ Categorical Data Analysis
df['column'].value_counts() → Frequency of each category.
df['column'].unique() → Unique categories.

6️⃣ Numerical Data Distribution
df.hist(figsize=(10, 5)) → Histograms for numerical columns.
sns.boxplot(data=df) → Box plots for outlier detection.
sns.pairplot(df) → Pairwise relationships.

7️⃣ Feature Engineering
Creating new columns: df['new_feature'] = df['col1'] * df['col2']
Binning: pd.cut(df['column'], bins=5)
Encoding categorical data: pd.get_dummies(df, columns=['category_col'])

8️⃣ Correlation Analysis
sns.heatmap(df.corr(), annot=True, cmap='coolwarm') → Correlation heatmap.

9️⃣ Outlier Detection
sns.boxplot(x=df['column']) → Detect outliers using box plots.
df[(df['column'] > upper_bound) | (df['column'] < lower_bound)] → Identify extreme values.

🔟 Time Series Analysis (if applicable)
df['date_column'] = pd.to_datetime(df['date_column'])
df.set_index('date_column').plot() → Plot time trends.


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Creating a simple dataset
data = {
    "Study Hours": [2, 4, 6, 8, 10],
    "Exam Score": [50, 65, 75, 85, 95],
    "Sleep Hours": [6, 7, 8, 7, 6],
    "Stress Level": [80, 70, 60, 50, 40]
}

df = pd.DataFrame(data)

# Compute correlation
correlation_matrix = df.corr()

# Plot heatmap
plt.figure(figsize=(5,4))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=1)
plt.title("Student Performance Correlation Heatmap")
plt.show()


In [ ]:
plt.figure(figsize=(7,5))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=1)
plt.title("Correlation Matrix Heatmap")
plt.show()

In [ ]:
! pip install seaborn

In [18]:
df_st_table1= pd.read_sql("SELECT count(*) FROM stock_prices", conn)
df_st_table1

,count(*)
0,5736819


In [19]:
df_st_table= pd.read_sql("SELECT * FROM stock_prices_backup_new", conn)
df_st_table

,id,symbol,date,open,high,low,close,volume,adjclose,ticker
0,1,20MICRONS.NS,2008-10-06,40.000000,40.000000,15.800000,16.775000,23501600.0,14.652864,20MICRONS.NS
1,2,20MICRONS.NS,2008-10-07,16.000000,19.000000,13.925000,15.025000,9113400.0,13.124249,20MICRONS.NS
2,3,20MICRONS.NS,2008-10-08,14.000000,14.600000,12.550000,13.250000,2464384.0,11.573796,20MICRONS.NS
3,4,20MICRONS.NS,2008-10-10,12.450000,12.450000,10.825000,11.600000,1207928.0,10.132533,20MICRONS.NS
4,5,20MICRONS.NS,2008-10-13,12.150000,13.300000,11.650000,12.325000,898692.0,10.765815,20MICRONS.NS
...,...,...,...,...,...,...,...,...,...,...
5736814,5736815,ZYDUSWELL.NS,2025-03-20,1624.949951,1667.150024,1624.949951,1650.449951,71909.0,1650.449951,ZYDUSWELL.NS
5736815,5736816,ZYDUSWELL.NS,2025-03-21,1661.550049,1676.949951,1645.349976,1668.800049,16485.0,1668.800049,ZYDUSWELL.NS
5736816,5736817,ZYDUSWELL.NS,2025-03-24,1668.800049,1672.949951,1649.599976,1667.599976,23153.0,1667.599976,ZYDUSWELL.NS
5736817,5736818,ZYDUSWELL.NS,2025-03-25,1675.000000,1688.750000,1651.000000,1680.150024,33812.0,1680.150024,ZYDUSWELL.NS


In [17]:
df_st_table.dtypes

symbol       object
date         object
open         object
high         object
low          object
close        object
volume       object
adjclose     object
dividends    object
ticker       object
splits       object
dtype: object

In [ ]:
cursor.execute("""CREATE TABLE stock_prices_backup_new (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    symbol TEXT,
    date TEXT NOT NULL,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    volume INTEGER,
    adjclose REAL,
    ticker TEXT NOT NULL,
    UNIQUE (ticker, date)  -- Enforcing uniqueness
);
""")
conn.commit()  # Commit the changes
